In [4]:
import os 

In [5]:
os.chdir("../")

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class TrainingConfig:
   
   root_dir: Path
   train_model_path: Path
   updated_model_base_path: Path
   training_data: Path
   params_epochs: int
   params_batch_size: int
   params_is_augmentation: bool
   param_image_size: list   

   
        

In [7]:
from src.cnnClassifier.constant import *
from src.cnnClassifier.utils.common import read_yaml, create_directories
from src.cnnClassifier.entity.config_entity import PrepareCallbacksConfig

d:\End-to-End chicken-diesase-implementation


In [8]:
class ConfigurationManager:
    
    def __init__(self, configfile = CONFIG_FILE_PATH, paramsfile = PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(configfile)
        self.params = read_yaml(paramsfile)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        
        config = self.config.prepare_callbacks
        
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        
        prepare_callbacks_config = PrepareCallbacksConfig(
            
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
            
        )
        
        return prepare_callbacks_config
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.root_dir, "Chiken-fecal-images")
        
        create_directories([
            Path(training.root_dir)
        ])
        
        training_config = TrainingConfig(
            
            root_dir=Path(training.root_dir),
            train_model_path=Path(training.train_model_path),
            updated_model_base_path=Path(prepare_base_model.updated_base_model_config),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            param_image_size=params.IMAGE_SIZE,
            
        )
        
        return training_config

In [12]:
import time
import tensorflow as tf

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig) -> None:
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timeStamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_log_at_{timeStamp}",
            
        ) 
        
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        
        return tf.keras.callbacks.ModelCheckpoint(
            self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        
        
    def _get_tb_ckpt_callbacks(self):
        
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
        
    
    

In [11]:

import time
import tensorflow as tf

In [13]:

class Training:
    def __init__(self, configfile: TrainingConfig) -> None:
        self.config = configfile
        
    def get_base_model(self):
        
        self.model = tf.keras.models.Model(
            self.config.updated_model_base_path
        )
        
    def train_valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20, 
        )
        
        dataflow_kwargs = dict(
            
            target_size = self.config.param_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
    
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False
            **dataflow_kwargs
        )
        
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.3
                **datagenerator_kwargs
            )
            
        else:
            train_datagenerator = valid_datagenerator
        
        
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
        
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
        
    def train(self, callbacks_list: list):
        self.steps_per_epochs = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epochs,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callbacks_list
        )
        
        self.save_model(
            path=self.config.train_model_path,
            model=self.model
        )
        

In [15]:
try:
    
    config = ConfigurationManager()
    prepare_callbacks = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(prepare_callbacks)
    callback_list = prepare_callbacks._get_tb_ckpt_callbacks()
    
    training_config = config.get_training_config()
    training = Training(training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callbacks_list=callback_list
    )
    
except Exception as e:
    raise




[2023-07-25 20:44:42,781 - INFO - common - yaml file: config\config.yaml loaded successfully]
[2023-07-25 20:44:42,781 - INFO - common - yaml file: params.yaml loaded successfully]
[2023-07-25 20:44:42,781 - INFO - common - Directory created ! artifacts]
[2023-07-25 20:44:42,791 - INFO - common - Directory created ! artifacts\prepare_callbacks\checkpoint_root_log_dir]
[2023-07-25 20:44:42,793 - INFO - common - Directory created ! artifacts\prepare_callbacks\tensorboard_root_log_dir]


AttributeError: 'ConfigurationManager' object has no attribute 'get_training_config'

In [16]:
c=ConfigurationManager()
c.get_training_config()

[2023-07-25 22:21:33,447 - INFO - common - yaml file: config\config.yaml loaded successfully]
[2023-07-25 22:21:33,449 - INFO - common - yaml file: params.yaml loaded successfully]
[2023-07-25 22:21:33,458 - INFO - common - Directory created ! artifacts]


AttributeError: 'ConfigurationManager' object has no attribute 'get_training_config'